## SILVER TO GOLD LAYER


In [1]:
import pandas as pd
from global_paths import GlobalPaths
from common_utilities import TradeHistory

### TradeHistory Paths

In [2]:
# Initialize TradeHistory Paths
TradeHistoryPath = GlobalPaths(source_name="DATA", object_name="TradeHistory")
TradeHistorySilverPath = TradeHistoryPath.createLayer(layer_name="SILVER")
TradeHistorySilverFile = TradeHistorySilverPath.joinpath("TradeHistory_data.csv")
TradeHistoryGoldPath = TradeHistoryPath.createLayer(layer_name="GOLD")
TradeHistoryGoldFile = TradeHistoryGoldPath.joinpath("TradeHistory_data.csv")

# Initialize Equity Paths
EquityPath = GlobalPaths(source_name="DATA", object_name="Equity")
EquitySilverPath = EquityPath.createLayer(layer_name="SILVER")
EquitySilverFile = EquitySilverPath.joinpath("Equity_data.csv")

### Gold Layer - Trade History


In [4]:
# read the csv file
df_TradeHistory = pd.read_csv(
    TradeHistorySilverFile #, convert_dates=False
)

# Filter Equity trades
df_TradeHistory = df_TradeHistory[df_TradeHistory["segment"] == "EQ"]

# replace scrip code to compnay name
df_Equity = pd.read_csv(EquitySilverFile)


# Merge df_TradeHistory with df_Equity on the matching columns


df_TradeHistory = df_TradeHistory.merge(
    df_Equity[["security_code", "security_id"]],
    left_on="scrip_code",
    right_on="security_code",
    how="left",
)


# Assign the new column 'stock_name' in df_TradeHistory to the values from 'security_id'


df_TradeHistory["stock_name"] = df_TradeHistory["security_id"]


# Combine 'date' and 'trade_time' into a single datetime column


# with fallback to 'date' if 'trade_time' is missing


# Convert the resulting column to datetime


df_TradeHistory["datetime"] = pd.to_datetime(
    df_TradeHistory["date"] + " " + df_TradeHistory["trade_time"].fillna("00:00:00"),
    format="%Y-%m-%d %H:%M:%S",
)


# sort the dataframe by date


df_TradeHistory = df_TradeHistory.sort_values(by="datetime")


df_TradeHistory["datetime"] = df_TradeHistory["datetime"].astype("str")


# rename the columns


df_TradeHistory = df_TradeHistory.rename(
    columns={
        "side": "trade_type",
        "quantity": "trade_quantity",
        "price": "trade_price",
    }
)


data_dict = list()


trade_history = dict()


for _, row in df_TradeHistory.iterrows():

    stock_name = row["stock_name"]

    if stock_name not in trade_history:

        trade_history[stock_name] = TradeHistory(stock_name)

    if row["trade_type"] == "Buy":

        row["buy_price"] = row["trade_price"]

        trade_history[stock_name].trade_price.append(row["trade_price"])

        trade_history[stock_name].trade_quantity.append(row["trade_quantity"])

    elif row["trade_type"] == "Sell":

        row["sell_price"] = row["trade_price"]

        row["buy_price"] = trade_history[stock_name].fifo_sell_calc(
            row["trade_quantity"]
        )

    else:

        raise Exception(f'{row["trade_type"]} was never excepected')

    row["holding_quantity"] = trade_history[stock_name].holding_quantity()

    row["avg_price"] = trade_history[stock_name].calc_avg_price()

    data_dict.append(row)


df_TradeHistory = pd.DataFrame(data_dict)


df_TradeHistory = df_TradeHistory.round(2)
df_TradeHistory = df_TradeHistory[
    [
        "datetime",
        "stock_name",
        "trade_type",
        "trade_quantity",
        "buy_price",
        "sell_price",
        "holding_quantity",
        "avg_price",
    ]
]


# Save the result as a csv file


gold_trade_history_file_path = gold_path.joinpath("TradeHistory.csv")


df_TradeHistory.to_csv(gold_trade_history_file_path, orient="records", indent=4)


df_TradeHistory.info()


print("Gold Layer csv file for trade history successfully created at:")


print(gold_trade_history_file_path.resolve())

<class 'pandas.core.frame.DataFrame'>
Index: 42 entries, 0 to 41
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   datetime          42 non-null     object 
 1   stock_name        42 non-null     object 
 2   trade_type        42 non-null     object 
 3   trade_quantity    42 non-null     float64
 4   buy_price         42 non-null     float64
 5   sell_price        15 non-null     float64
 6   holding_quantity  42 non-null     float64
 7   avg_price         42 non-null     float64
dtypes: float64(5), object(3)
memory usage: 4.0+ KB
Gold Layer csv file for trade history successfully created at:
C:\Users\prashant.tripathi\Code\Unilever\Upstox\DATA\GOLD\TradeHistory.csv
